In [1]:
import napari_toska as nts

In [2]:
from skimage.morphology import skeletonize
from skimage.data import cells3d
from skimage.filters import threshold_otsu
from scipy.ndimage import binary_fill_holes
from skimage.measure import label
import numpy as np
import napari

In [3]:
cells3d().shape

(60, 2, 256, 256)

In [4]:
nuclei_3d = cells3d()[:,1,:,:]

In [5]:
thresh_nuclei = binary_fill_holes(nuclei_3d > threshold_otsu(nuclei_3d))

In [6]:
labels = label(thresh_nuclei)

In [7]:
label_scalars = np.unique(labels)

In [8]:
sizes = [np.sum(labels[labels==i]) for i in label_scalars]

In [9]:
for index,size in enumerate(sizes):
    if size < 10000:
        labels[labels==index] = 0

In [12]:
viewer = napari.Viewer()

In [13]:
viewer.add_labels(labels)

<Labels layer 'labels' at 0x1b9482c40>

In [15]:
labeled_skeletons = nts.generate_labeled_skeletonization(labels)
viewer.add_labels(labeled_skeletons)

<Labels layer 'labeled_skeletons' at 0x1c29e9fa0>

In [19]:
from napari_toska._backend_toska_functions import n26_parse_skel_3d

In [ ]:
def categorize_skeleton_components(labeled_skeletons:'napari.types.LabelsData'):
    
    coords, e_pts, b_pts, brnch, brnch_ids, brnch_lengths = n26_parse_skel_3d(labeled_skeletons)
    
    return

In [24]:
def n26_parse_skel_3d(skel, z_dir: int = 0, y_dir: int = 1, x_dir: int = 2):
    
    '''Parameters:
       -----------
       
       skel:
       z_dir:
       y_dir:
       x_dir:
       
       Returns:
       --------
       
       coords:
       e_pts:
       b_pts:
       brnch:
       brnch_ids:
       brnch_lengths:'''
    
    coords = np.asarray(np.where(skel)).T
    e_pts, b_pts = n26_pt_classification(skel, coords, z_dir, y_dir, x_dir)
    
    brnch = np.copy(skel)
    for i in b_pts:
        brnch[i[0],i[1],i[2]] = 0
        
    brnch, brnch_ids = label(brnch, structure = np.ones((3,3,3), dtype = int))
    brnch_ids = tuple(np.arange(1, brnch_ids+1))
    brnch_lengths = tuple([np.sum(brnch[brnch == i].astype(bool).astype(np.uint16)) for i in brnch_ids])
    
    return coords, e_pts, b_pts, brnch, brnch_ids, brnch_lengths

In [21]:
label_129 = labeled_skeletons == 129

In [20]:
import pandas as pd

In [ ]:
def classify_skeleton_coordinates(skeleton):
    
    coordinates = np.asarray(np.where(skeleton>0)).T
    end_points, branching_points = n26_pt_classification(skeleton, coordinates)
    
    df = pd.DataFrame(coordinates, columns=['z','y','x'])
    df['is_endpoint']=False
    df.loc[end_points, 'is_endpoint']=True
    
    return 

In [ ]:
def label_branches(skeleton, branch_points):
    
    return